In [1]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
sales = pd.read_csv(r'd:\DownLoads\foodmart.sales.tsv', sep='\t',header=0,parse_dates=[2])
products = pd.read_csv(r'd:\DownLoads\foodmart.products.tsv', sep='\t',header=0)

In [3]:
sales = sales.merge(products[['product_id','product_name']], on=['product_id'], how='inner')

In [4]:
sales.head()

,product_id,store_id,date,sales,product_name
0,4,6,1997-01-01,4,Washington Cream Soda
1,4,7,1997-01-05,3,Washington Cream Soda
2,4,6,1997-01-06,2,Washington Cream Soda
3,4,17,1997-01-11,2,Washington Cream Soda
4,4,24,1997-01-11,2,Washington Cream Soda


In [5]:
sales_sparse = pd.pivot_table(sales,values='sales',index=['date','store_id'],columns=['product_name'],fill_value=0,aggfunc=lambda x: x)

In [6]:
sales_sparse.head(15)

product_name         ADJ Rosy Sunglasses  Akron City Map  \
date       store_id                                        
1997-01-01 6                           0               0   
           14                          0               0   
1997-01-02 11                          0               0   
           23                          0               0   
1997-01-03 7                           0               0   
           23                          0               0   
1997-01-04 14                          0               0   
1997-01-05 2                           0               0   
           7                           0               0   
           15                          0               0   
           23                          0               0   
1997-01-06 3                           0               0   
           6                           0               0   
           24                          0               0   
1997-01-07 11                          0               0   

product_name         Akron Eyeglass Screwdriver  American Beef Bologna  \
date       store_id                                                      
1997-01-01 6                                  0                      0   
           14                                 0                      0   
1997-01-02 11                                 0                      0   
           23                                 0                      0   
1997-01-03 7                                  0                      0   
           23                                 0                      0   
1997-01-04 14                                 0                      0   
1997-01-05 2                                  0                      0   
           7                                  0                      0   
           15                                 0                      0   
           23                                 0                      0   
1997-01-06 3                                  0                      0   
           6                                  4                      0   
           24                                 3                      0   
1997-01-07 11                                 0                      0   

product_name         American Chicken Hot Dogs  American Cole Slaw  \
date       store_id                                                  
1997-01-01 6                                 0                   0   
           14                                0                   0   
1997-01-02 11                                0                   0   
           23                                0                   0   
1997-01-03 7                                 0                   0   
           23                                0                   0   
1997-01-04 14                                0                   0   
1997-01-05 2                                 0                   0   
           7                                 0                   0   
           15                                0                   0   
           23                                0                   0   
1997-01-06 3                                 0                   0   
           6                                 0                   0   
           24                                0                   2   
1997-01-07 11                                0                   0   

product_name         American Corned Beef  American Foot-Long Hot Dogs  \
date       store_id                                                      
1997-01-01 6                            0                            0   
           14                           0                            0   
1997-01-02 11                           0                            0   
           23                           0                            0   
1997-01-03 7                            0                            0   
           

In [7]:
corr_data = []
for i, lhs_col in enumerate(sales_sparse.columns):
    for j, rhs_col in enumerate(sales_sparse.columns):
        if i >= j:
            continue
            
        corr,p = pearsonr(sales_sparse[lhs_col],sales_sparse[rhs_col])
        corr_data.append([lhs_col,rhs_col,corr,p])

In [8]:
sales_correlations = pd.DataFrame.from_records(corr_data)

In [9]:
sales_correlations.columns = ['prod_A','prod_B','corr','p']

In [10]:
sales_correlations.head()

,prod_A,prod_B,corr,p
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484


In [11]:
#METHOD XOLMA

In [12]:
rejected, p_corrected, a1,a2 = multipletests(sales_correlations.p,alpha=0.05,method='holm')

In [13]:
sales_correlations['p_corrected'] =p_corrected
sales_correlations['rejected'] = rejected

In [14]:
sales_correlations.head()

,prod_A,prod_B,corr,p,p_corrected,rejected
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,1.0,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,1.0,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,1.0,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,1.0,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,1.0,False


In [15]:
sales_correlations[sales_correlations.rejected == True].sort_values(by='corr',ascending=False).head()

,prod_A,prod_B,corr,p,p_corrected,rejected
1063670,Just Right Vegetable Soup,Plato French Roast Coffee,0.340598,1.226033e-22,1.488970e-16,True
885574,Great Muffins,Nationeel Grape Fruit Roll,0.322176,2.688803e-20,3.265443e-14,True
473067,Club Low Fat Cottage Cheese,Skinner Strawberry Drink,0.306701,1.883995e-18,2.288034e-12,True
1181001,Robust Monthly Home Magazine,Tri-State Lemons,0.303269,4.674973e-18,5.677558e-12,True
1160248,Pleasant Regular Ramen Soup,Shady Lake Ravioli,0.298502,1.619119e-17,1.966350e-11,True


In [16]:
#Method XOXBERGA

In [17]:
rejected, p_corrected, a1,a2 = multipletests(sales_correlations.p,alpha=0.05,method='fdr_bh')

In [18]:
sales_correlations['p_corrected'] =p_corrected
sales_correlations['rejected'] = rejected

In [19]:
sales_correlations.head()

,prod_A,prod_B,corr,p,p_corrected,rejected
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,0.203716,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,0.956078,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,0.630699,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,0.600790,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,0.541916,False


In [20]:
sales_correlations.rejected.value_counts()

False    1138407
True       76054
Name: rejected, dtype: int64

In [21]:
sales_correlations[sales_correlations.rejected == True].sort_values(by='corr',ascending=False).head()

,prod_A,prod_B,corr,p,p_corrected,rejected
1063670,Just Right Vegetable Soup,Plato French Roast Coffee,0.340598,1.226033e-22,1.488970e-16,True
885574,Great Muffins,Nationeel Grape Fruit Roll,0.322176,2.688803e-20,1.632723e-14,True
473067,Club Low Fat Cottage Cheese,Skinner Strawberry Drink,0.306701,1.883995e-18,7.626793e-13,True
1181001,Robust Monthly Home Magazine,Tri-State Lemons,0.303269,4.674973e-18,1.419393e-12,True
1160248,Pleasant Regular Ramen Soup,Shady Lake Ravioli,0.298502,1.619119e-17,3.932713e-12,True
